## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")

### LCEL(LangChain Expression Language)
- 一个最简单示例
- One of the simplest examples
<hr>

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于 {topic}的笑话")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰激凌"})

'一天，一只蚂蚁问一只蜜蜂：“你知道冰激凌为什么不能做家务吗？”\n蜜蜂摇头说不知道。\n蚂蚁笑着说：“因为它总是融化了！”'

- Prompt

In [3]:
prompt_value = prompt.invoke({"topic": "刺猬"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='给我讲一个关于 刺猬的笑话')])

In [4]:
prompt_value.to_messages()

[HumanMessage(content='给我讲一个关于 刺猬的笑话')]

In [5]:
prompt_value.to_string()

'Human: 给我讲一个关于 刺猬的笑话'

- model

In [6]:
message = model.invoke(prompt_value)
message

AIMessage(content='一天，一只乌龟问一只刺猬：“你为什么那么爱打滚呢？”\n\n刺猬回答：“因为我不想步步皆针。”\n\n乌龟一愣，然后笑了：“哈哈，原来你是想轻松一点啊。”\n\n刺猬笑着摇了摇头：“不，是因为我想变成一个滚刺球。”', response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 22, 'total_tokens': 142, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-06048dcf-c7ca-4a36-8fa5-9413c79bac4e-0')

- 使用llm的区别
- Differences in the use of llm

In [7]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nAI:为什么刺猬总是带着刺？因为它们害怕被人捉弄啊！'

- Output parser

In [8]:
output_parser.invoke(message)

'一天，一只乌龟问一只刺猬：“你为什么那么爱打滚呢？”\n\n刺猬回答：“因为我不想步步皆针。”\n\n乌龟一愣，然后笑了：“哈哈，原来你是想轻松一点啊。”\n\n刺猬笑着摇了摇头：“不，是因为我想变成一个滚刺球。”'

LCEL的Pipeline：
- ![Alt Text](asset/lcel01.png)

### RAG Search Exampl
- 建立向量数据
- 使用RAG增强
### RAG Search Exampl
- Building Vector Data
- Enhancement with RAG
<hr>

In [9]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [10]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

- 自定义也非常简单


In [11]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [12]:
chain.invoke({"question": "where did harrison work", "language": "chinese"})

'Harrison在肯舒工作。'